### Load Colab Notebook settings

**Link Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/LEO/'  #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Try GPU power**

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=ea40ede1ae686158192b9e3c5af0775b4a85864c1f51924170175d8b9aa7fc1e
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.6 GB  | Proc size: 454.7 MB
GPU RAM Free: 11372MB | Used: 69MB | Util   1% | Total 11441MB


In [ ]:
#!kill -9 -1

In [ ]:
import os
import sys

In [7]:
# unet.py
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Convolution Blocks
def convolution_down(prev_layer, nb_channels, acti='relu'):
    kernel_size = 3
    conv = Conv2D(nb_channels, kernel_size, activation=acti, padding='same', kernel_initializer='he_normal')(prev_layer)
    conv = BatchNormalization()(conv)
    conv = Conv2D(nb_channels, kernel_size, activation=acti, padding='same', kernel_initializer='he_normal')(conv)
    conv = BatchNormalization()(conv)
    pool = MaxPooling2D(pool_size=(2, 2))(conv)
    return conv, pool
    
def convolution_up(prev_layer, nb_channels, layer_merging, acti='relu'):
    kernel_size = 3
    up = Conv2D(nb_channels, 2, activation=acti, padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(prev_layer))
    merged = concatenate([layer_merging, up], axis=3)
    conv = Conv2D(nb_channels, kernel_size, activation=acti, padding='same', kernel_initializer='he_normal')(merged)
    conv = BatchNormalization()(conv)
    conv = Conv2D(nb_channels, kernel_size, activation=acti, padding='same', kernel_initializer='he_normal')(conv)
    conv = BatchNormalization()(conv)
    return conv

# U-Net model for 256x256 images
def unet256(input_size, lr=0.005, verbose=True):
    inputs = Input(shape=input_size)
    kernel_size = 3

    # Creation of the layers
    conv1, pool1 = convolution_down(inputs, 64)
    conv2, pool2 = convolution_down(pool1, 128)
    conv3, pool3 = convolution_down(pool2, 256)
    conv4, pool4 = convolution_down(pool3, 512)
    conv5, _ = convolution_down(pool4, 1024)
    
    conv6 = convolution_up(conv5, 512, conv4)
    conv7 = convolution_up(conv6, 256, conv3)
    conv8 = convolution_up(conv7, 128, conv2)
    conv9 = convolution_up(conv8, 64, conv1)
    
    conv10 = Conv2D(2, kernel_size, activation='relu', padding='same', kernel_initializer = 'he_normal')(conv9)
    conv11 = Conv2D(1, 1, activation = 'sigmoid')(conv10)
    
    model = Model(inputs = inputs, outputs = conv11)
    
    model.compile(optimizer = Adam(lr=lr), loss = 'binary_crossentropy', metrics = ['acc', f1_m])
    
    if(verbose == True):
        model.summary()

    return model

Using TensorFlow backend.


In [ ]:
# fit_model.py
def fit_unet(rotation, MODEL_PATH, MODEL_NAME):
    # Load dataset
    data_gen_args = dict(rotation_range=rotation,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

    train_generator = dataGenerator(2, DATA_PATH+'train',
                                    'img_aug_tr',
                                    'gt_aug_tr'
                                    ,data_gen_args,
                                    (SIDE,SIDE))

    validation_generator = dataGenerator(2, DATA_PATH+'train',
                                    'img_aug_te',
                                    'gt_aug_te'
                                    ,data_gen_args,
                                    (SIDE,SIDE))

    filepath = "weights.{epoch:02d}-{val_f1_m:.2f}.hdf5"

    csv_logger = CSVLogger("AccuracyHistory.csv")
    cp_callback = ModelCheckpoint(filepath=filepath, verbose=1, save_weights_only=True, period=1)
    
    # Load and fit the model
    model = unet256((SIDE,SIDE,3),lr=0.001, verbose=False)
    model.fit_generator(generator=train_generator,
                    steps_per_epoch=200,#2000
                    epochs=110, #10
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps = 70,#700
                    validation_freq=1,
                    initial_epoch=0,
                    callbacks=[cp_callback, csv_logger])
    
    return model

In [ ]:
# data_postprocess
import os, re, sys, csv
import skimage.io as io
import numpy as np
import matplotlib.image as mpimg
from skimage import img_as_ubyte
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array


def save_model(model, save_path, output_filename):
    #save_path+='/'
    print(save_path)
    #if not os.path.exists(save_path):
    #    print("here")
    #    os.mkdir(save_path)
    #    print("folder created")
    model_json = model.to_json()
    with open(save_path+output_filename + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(save_path+output_filename + ".h5")
    print("Saved model to disk")

def load_model(path):
    #'model_saved/Model_UNET_256_acc99_valid.json'
    json_file = open(path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    return model

def reshape_img(path_from, path_to, shape):
    if not os.path.exists(path_to):
        os.mkdir(path_to)
    filenames = os.listdir(path_from)
    for i, fileNb in enumerate(filenames):
        if(fileNb!='.DS_Store'):        
            im1 = Image.open(path_from+fileNb)
            # use one of these filter options to resize the image
            im2 = im1.resize((shape, shape), Image.NEAREST)
            im2.save(path_to+fileNb)
            
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def savePredictedImages(from_path, save_path, predictions, concat=True):
    #if os.path.exists(save_path):
    #    shutil.rmtree(save_path)
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    list_im_name = sorted_aphanumeric(os.listdir(from_path))
    if(list_im_name[0]=='.DS_Store'):
        list_im_name=list_im_name[1:]
    for name, pred in zip(list_im_name, predictions):
        img = mpimg.imread(from_path+name)
        if(concat):
            if(len(pred.shape)==3 and pred.shape[2]==1):
                pred = pred.reshape(pred.shape[0],pred.shape[1])
            cimg = concatenate_images(img, pred)
            io.imsave(save_path+"/prediction"+name[4:], img_as_ubyte(cimg))
        else:
            io.imsave(save_path+"/prediction"+name[4:], img_as_ubyte(pred))
    print("save pred ok")
            
def data_load_for_prediction(directory_name, RGB_image=False): 
    filenames = sorted_aphanumeric(os.listdir(directory_name))
    imgs = []
    i=0
    nb_imgs = len(filenames)
    for fileNb in filenames:
        if(fileNb!='.DS_Store'):  
            full_name = directory_name+fileNb
            img=mpimg.imread(full_name)
            if RGB_image:
                imgr = img_to_array(img)
            else:
                imgr = img_to_array(img).reshape(SIDE_FINAL, SIDE_FINAL)
            imgs.append(imgr)
            i+=1
            sys.stdout.write("\rImage {}/{} is being loaded".format(i,nb_imgs))
            sys.stdout.flush()
        else:
            nb_imgs-=1
    print()

    return np.asarray(imgs)

def average_image(IMGS_weighted_folders):
    total = IMGS_weighted_folders[0][0]
    means = IMGS_weighted_folders[0][1] * total
    print('Size should be (50, 608, 608) and currently is:', means.shape) 
    for i in range(1,len(IMGS_weighted_folders)):
        tupl = IMGS_weighted_folders[i]
        weight = tupl[0]
        total+=weight
        for j, img in enumerate(tupl[1]):
            means[j]+=img*weight
    return means/total

def color_patch(patch, thresh=0.25):
    m = np.mean(patch)
    if(m>thresh):
        return 1, np.ones(16*16).reshape(16,16)
    else:
        return 0, np.zeros(16*16).reshape(16,16)

def color_patch_full(img):
    vs=[]
    patchs=[]
    img_patches = img_crop(img, PATCH_SIZE, PATCH_SIZE)
    for i in range(len(img_patches)):
        v, X = color_patch(img_patches[i])
        vs.append(v)
        patchs.append(X)
    patchs = np.asarray(patchs)
    return vs, patchs

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

def patched_imgs_and_vs(means):
    vs_for_each_imgs = []
    imgs_means_patched = []
    for index, mean_img in enumerate(means):
        vs, patchs = color_patch_full(mean_img)
        vs_for_each_imgs.append(vs)
        
        nb_patch_per_side = int(SIDE_FINAL/PATCH_SIZE)
        patchs = patchs.reshape(nb_patch_per_side, nb_patch_per_side, PATCH_SIZE, PATCH_SIZE)
        
        for j in range(0,nb_patch_per_side):
            tmp = patchs[j,0]
            for i in range(1,nb_patch_per_side):
                tmp = np.concatenate((tmp, patchs[j,i]), axis=1) 
            if(j==0):
                TMP = tmp
            else:
                TMP = np.concatenate((TMP, tmp), axis=0)
        imgs_means_patched.append(np.rot90(np.rot90(np.rot90(np.flip(TMP, 0)))))
        sys.stdout.write("\rImage {}/{} is being processed".format(index+1,len(means)))
        sys.stdout.flush()
    return imgs_means_patched, vs_for_each_imgs

def create_csv_submission(imgs, vs_for_each_imgs, name):
    labels = np.arange(0, SIDE_FINAL, PATCH_SIZE)
    nb_patch_per_side = int(SIDE_FINAL/PATCH_SIZE)
    with open(name, 'w') as csvfile:
        fieldnames = ['id', 'prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for nb, img in enumerate(imgs):
            ID = str(nb+1).zfill(3)
            row = 0
            col = 0
            for v in vs_for_each_imgs[nb]:
                col_ = col%nb_patch_per_side
                ID2 = ID+'_'+str(labels[row])+'_'+str(labels[col_])    
                writer.writerow({'id':ID2,'prediction':v})
                if(col == 37):
                    col=0
                    row+=1
                else:
                    col+=1

In [ ]:
# data_preprocess.py
import sys, re, os, shutil
import matplotlib.image as mpimg
import numpy as np
from keras.models import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from PIL import Image

def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return np.asarray(sorted(data, key=alphanum_key))

def data_load(directory_name): 
    filenames = sorted_aphanumeric(os.listdir(directory_name))
    imgs = []
    i = 0
    nb_imgs = len(filenames)
    for j, fileName in enumerate(filenames):
        if(fileName[0]=='.'):
            nb_imgs-=1
        else:
            full_name = directory_name+fileName
            img=mpimg.imread(full_name)
            imgr = img_to_array(img)
            imgs.append(imgr)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,nb_imgs))
            sys.stdout.flush()
            i+=1
    print()
    return np.asarray(imgs)

def dataGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, target_size,
                  image_color_mode = "rgb", mask_color_mode = "grayscale",
                  image_save_prefix  = "image",mask_save_prefix  = "mask",
                  flag_multi_class = False,num_class = 2,save_to_dir = None, seed = 1):

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen  = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    
    for (img,mask) in train_generator:
        if(np.max(img) > 1):
            img = img / 255
            mask = mask / 255
            mask[mask > 0.5] = 1
            mask[mask <= 0.5] = 0
        yield (img,mask)
        
def create_validation_train_directory(path, dir_images, dir_labels, i, seed):
    # 0<=i<5 
    origin_dirs = [dir_images, dir_labels]
    for name in origin_dirs:
        filenames = sorted_aphanumeric(os.listdir(path+name))
        if('.DS_Store' in filenames):
            filenames = np.delete(filenames, 0)
        np.random.seed(seed)
        permut = np.random.permutation(len(filenames))
        index = int(len(permut)*0.2)
        test_ind = permut[i*index:(i+1)*index]
        test_filenames = filenames[test_ind]
        trai_ind = permut[index:]
        trai_filenames = filenames[trai_ind]
        new = [name+'_te', name+'_tr']
        for n in new:  
            if os.path.exists(path+n):
                shutil.rmtree(path+n)
            os.mkdir(path+n)
        for test_f in test_filenames:
            img = Image.open(path+name+'/'+test_f)
            img.save(path+new[0]+'/'+test_f)
        for trai_f in trai_filenames:
            img = Image.open(path+name+'/'+trai_f)
            img.save(path+new[1]+'/'+trai_f)
        for n in new:
            print(len(os.listdir(path+n)))

In [ ]:
def dataset_augmentation(directory_from, directory_to): 
    if not os.path.exists(directory_to):
        os.mkdir(directory_to)
    datagen = ImageDataGenerator()
    filenames = os.listdir(directory_from)
    #create 24 rotated images for one image
    angle = 45
    zoom = 0.75
    imgs = []
    
    for i, fileNb in enumerate(filenames):
        if fileNb != ".DS_Store":
            full_name = directory_from+fileNb
            img=mpimg.imread(full_name)
            io.imsave(directory_to+fileNb, img)
            img_asarray = img_to_array(img)

            out = datagen.apply_transform(x=img_asarray, transform_parameters={'theta':angle, 'zx':zoom, 'zy':zoom})
            j = 100+i
            fileNb2 = "satImage_%03d"%j + ".png"
            io.imsave(directory_to+fileNb2, out)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
            sys.stdout.flush()

In [16]:
dataset_augmentation(root_path+"data/train/images/", root_path+"data/train/img_aug/")
dataset_augmentation(root_path+"data/train/groundtruth/", root_path+"data/train/gt_aug/")


Image 1/101 is being loaded

Image 2/101 is being loaded

Image 3/101 is being loaded

Image 4/101 is being loaded

Image 5/101 is being loaded

...

Image 97/101 is being loaded

Image 98/101 is being loaded

Image 99/101 is being loaded

Image 100/101 is being loaded

Image 101/101 is being loaded

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger
# --- Choose whether train a new model (1) or load a pretrained model
train_model = 1

# --- Indicate the path of the pretrained model (if it exists) and the images to load
#WEIGHT_PATH     =  # A remplir
DATA_PATH       = root_path+"data/"
DATA_TEST_PATH  = root_path+"data/test/"
MODEL_PATH      = root_path+"models/"
PRED_PATH       = DATA_PATH+"predictions/"

SIDE = 256
SIDE_FINAL = 608
PATCH_SIZE = 16
TRESHOLD = 0.2
K_FOLD = 5

In [25]:
# --- Cross validation

k_fold = 2
rotation_angles = [5,90]
for k in range(k_fold):
    
    # Creates a list to store the predictions 
    IMGS = []

    # Create the k folds
    create_validation_train_directory(DATA_PATH+'train/', 'img_aug', 'gt_aug', 1, 1)
    
    print("create k folds: done")
    for i in rotation_angles:
        # --- Train or load a pretrained model
        if train_model:
            MODEL_NAME = f'Model_k{k}_rot{i}'
            model = fit_unet(i, MODEL_PATH, MODEL_NAME)
            print('fit model: done')
            # Save the trained model
            if not os.path.exists(MODEL_PATH):
                  os.mkdir(MODEL_PATH)

            save_model(model, MODEL_PATH+"model_saved", MODEL_NAME)

        else:
            # Load pretrained weights
            model = unet256(input_size = (SIDE,SIDE,3), verbose=False)
            #WEIGHT_PATH = ????
            model.load_weights(WEIGHT_PATH)

        # --- Load the testing set
        data_test = data_load(DATA_PATH+'test_resized/')
        print("load dataset: done")

        test_datagenerator  = ImageDataGenerator()
        testGene            = test_datagenerator.flow(data_test, batch_size=1)
        print("datagenerator: done")

        # --- Predict the the segmentation of the testing images
        # Reshape the images to 256x256 px and predict the segmentation
        reshape_img(DATA_PATH+"test/", DATA_PATH+"test_resized/", SIDE)
        results = model.predict(data_test,verbose=1)
        print("reshaped: done")

        # Create a prediction folder if it doesn't exist
        if not os.path.exists(PRED_PATH):
                os.mkdir(PRED_PATH)

        # Save the predictions
        PRED_FILE = PRED_PATH + f'Model_k{k}_rot{i}/'
        savePredictedImages(DATA_PATH+"test_resized/", 
                            PRED_FILE, 
                            results, 
                            concat=False)
        print("pred: done")

        # Resize the prediction to their final size (608x608 px)
        reshape_img(PRED_FILE, PRED_FILE, SIDE_FINAL)
        print("yolo")
        # Add the reshaped prediction to the prediction list
        IMGS.append((1,data_load_for_prediction(PRED_FILE)))


filenames ['satImage_001.png' 'satImage_002.png' 'satImage_003.png'
 'satImage_004.png' 'satImage_005.png' 'satImage_006.png'
 'satImage_007.png' 'satImage_008.png' 'satImage_009.png'
 'satImage_010.png' 'satImage_011.png' 'satImage_012.png'
 'satImage_013.png' 'satImage_014.png' 'satImage_015.png'
 'satImage_016.png' 'satImage_017.png' 'satImage_018.png'
 'satImage_019.png' 'satImage_020.png' 'satImage_021.png'
 'satImage_022.png' 'satImage_023.png' 'satImage_024.png'
 'satImage_025.png' 'satImage_026.png' 'satImage_027.png'
 'satImage_028.png' 'satImage_029.png' 'satImage_030.png'
 'satImage_031.png' 'satImage_032.png' 'satImage_033.png'
 'satImage_034.png' 'satImage_035.png' 'satImage_036.png'
 'satImage_037.png' 'satImage_038.png' 'satImage_039.png'
 'satImage_040.png' 'satImage_041.png' 'satImage_042.png'
 'satImage_043.png' 'satImage_044.png' 'satImage_045.png'
 'satImage_046.png' 'satImage_047.png' 'satImage_048.png'
 'satImage_049.png' 'satImage_050.png' 'satImage_051.png'
 'sa

OSError: ignored

In [ ]:
# --- Compute the mean of the predicted images
means = average_image(IMGS)

# --- Create submission in a .csv file
#if not os.path.exists(PATH_ROOT+"submissions/"):
#        os.mkdir(PATH_ROOT+"submissions/")

#create_csv_submission(patched_imgs, vs, PATH_ROOT+'submissions/Model_UNET_256_k_fold_0_f1_874a.csv')

In [25]:
# --- Cross validation step by step

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

k_fold = 5
rotation_angles = [5,90]

k=1
    
# Creates a list to store the predictions 
IMGS = []

# Create the k folds
create_validation_train_directory(DATA_PATH+'train/', 'img_aug', 'gt_aug', 1, 1)



44
176
40
160


In [26]:
print("create k folds: done")
for i in rotation_angles:
    # --- Train or load a pretrained model
    if train_model:
        MODEL_NAME = f'Model_k{k}_rot{i}'
        model = fit_unet(i, MODEL_PATH, MODEL_NAME)
        print('fit model: done')
        # Save the trained model
        if not os.path.exists(MODEL_PATH):
              os.mkdir(MODEL_PATH)

        save_model(model, MODEL_PATH, MODEL_NAME)

    else:
        # Load pretrained weights
        model = unet256(input_size = (SIDE,SIDE,3), verbose=False)
        #WEIGHT_PATH = ????
        model.load_weights(WEIGHT_PATH)



create k folds: done
Epoch 1/110
Found 44 images belonging to 1 classes.
Found 176 images belonging to 1 classes.
Found 40 images belonging to 1 classes.
Found 160 images belonging to 1 classes.
200/200 [==============================] - 117s 587ms/step - loss: 0.5372 - acc: 0.7901 - f1_m: 0.0137 - val_loss: 0.6260 - val_acc: 0.8042 - val_f1_m: 4.0789e-04

Epoch 00001: saving model to weights.01-0.00.hdf5
Epoch 2/110
200/200 [==============================] - 109s 547ms/step - loss: 0.5024 - acc: 0.8035 - f1_m: 9.0959e-05 - val_loss: 0.5508 - val_acc: 0.8029 - val_f1_m: 2.3939e-06

Epoch 00002: saving model to weights.02-0.00.hdf5
Epoch 3/110
200/200 [==============================] - 109s 547ms/step - loss: 0.5047 - acc: 0.7998 - f1_m: 1.8802e-05 - val_loss: 0.5546 - val_acc: 0.8026 - val_f1_m: 0.0000e+00

Epoch 00003: saving model to weights.03-0.00.hdf5
Epoch 4/110
200/200 [==============================] - 109s 546ms/step - loss: 0.5094 - acc: 0.8001 - f1_m: 0.0010 - val_loss: 0.55

FileNotFoundError: ignored

In [22]:


save_model(model, MODEL_PATH, MODEL_NAME)

gdrive/My Drive/LEO/models/
Saved model to disk


In [ ]:
    # --- Load the testing set
    data_test = data_load(DATA_PATH+'test_resized/')
    print("load dataset: done")

    # --- Predict the the segmentation of the testing images
    # Reshape the images to 256x256 px and predict the segmentation
    reshape_img(DATA_PATH+"test/", DATA_PATH+"test_resized/", SIDE)
    results = model.predict(data_test,verbose=1)
    print("reshaped: done")

    # Create a prediction folder if it doesn't exist
    if not os.path.exists(PRED_PATH):
            os.mkdir(PRED_PATH)

    # Save the predictions
    PRED_FILE = PRED_PATH + f'Model_k{k}_rot{i}/'
    savePredictedImages(DATA_PATH+"test_resized/", 
                        PRED_FILE, 
                        results, 
                        concat=False)
    print("pred: done")

    # Resize the prediction to their final size (608x608 px)
    reshape_img(PRED_FILE, PRED_FILE, SIDE_FINAL)
    print("yolo")
    # Add the reshaped prediction to the prediction list
    IMGS.append((1,data_load_for_prediction(PRED_FILE)))
